In [123]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [141]:
###########################
###########################
###########################
###### Data Cleaning ######
###########################
###########################
###########################


data = pd.read_excel('download.xls') # openning file

data = data[4:] #dropping garbage
data.loc[4] = data.loc[4].fillna(method = 'ffill')
data.loc[4] = data.loc[4]+data.loc[5]
data = data.drop(5)
data = data.drop(columns = data.columns[0])

data_transp = pd.DataFrame() # creating a reindexed by time dataframe
data_transp['time'] = 0
for i,j in enumerate(data.iloc[0].values[1:]):
    data_transp.loc[i, 'time'] = j
    
for i,j in enumerate(data[data.columns[0]][1:].values): # taking columns from the original dataframe
    data_transp[j] = data.iloc[i+1].values[1:]
    
data_transp = data_transp.replace(to_replace='---', value=np.nan)# replace nans

data_transp.columns = list(map(lambda x: x.strip() , data_transp.columns.values ))

# selecting relevant columns and renaming
names = {'time':'time','Gross domestic product':'gdp', 'Personal consumption expenditures': 'c' ,
         'Gross private domestic investment' : 'investment' , 'Fixed investment' : 'fixedi',
         'Nonresidential': 'nonresidential' , 'Structures' : 'structures', 'Equipment' :'equipment',
        'Intellectual property products':'intellectual','Residential':'residential' }

data_transp = data_transp[list(names.keys())] # selecting subset of columns
data_transp.columns = list(map( names.get, data_transp.columns))
 # renaming them, luckily order is matched, so no need in dict

In [142]:
data_transp.head()

,time,gdp,c,investment,fixedi,nonresidential,structures,equipment,intellectual,residential
0,1947Q1,12.552,11.583,8.319,8.923,6.065,28.558,4.427,1.374,27.629
1,1947Q2,12.519,11.776,7.681,8.692,5.971,28.407,4.332,1.357,26.248
2,1947Q3,12.493,11.814,7.455,8.973,5.861,28.600,4.186,1.356,30.261
3,1947Q4,12.688,11.818,8.898,9.711,6.024,28.189,4.429,1.328,36.094
4,1948Q1,12.879,11.877,9.811,10.022,6.374,28.774,4.804,1.340,35.623


In [143]:
###########################
###########################
###########################
######## Computing ########
###########################
###########################
###########################

data_transp[data_transp.columns[1:]] = data_transp[data_transp.columns[1:]].apply(lambda x: np.log(x)) # taking logs

names_dev =[]
names_growth = []
for i in data_transp.columns[1:]:
    
    # HP filter stuff
    X = data_transp[i]
    cycle, trend = sm.tsa.filters.hpfilter(X,1600)
    data_transp[i + '_HP_dev']= cycle
    names_dev.append(i + '_HP_dev')
    data_transp[i + '_diff'] = X.diff()
    # Growth stuff
    names_growth.append(i + '_diff')


In [144]:
###########################
###########################
###########################
######### Output  #########
###########################
###########################
###########################

# Cyclical correlation between variables

# We see that investment is very correlated with gdp
# Also fluctuations in investment are strongly procyclical

data_transp[names_dev].corr()['gdp_HP_dev']

gdp_HP_dev               1.000000
c_HP_dev                 0.768779
investment_HP_dev        0.837301
fixedi_HP_dev            0.808000
nonresidential_HP_dev    0.774688
structures_HP_dev        0.453605
equipment_HP_dev         0.815659
intellectual_HP_dev      0.383267
residential_HP_dev       0.514606
Name: gdp_HP_dev, dtype: float64

In [145]:
# Standard deviations of detrended log series

# We see that investment is very volatile compated to gdp and consumption
# It fluctuates more than 4 times stronger than gdp and consumption
# Also we see that residential investment is by far the most
# volatile component of investment. All the facts are consistent
# with the information on the slides

data_transp[names_dev].std()/data_transp[names_dev].std()[0]

gdp_HP_dev               1.000000
c_HP_dev                 0.766212
investment_HP_dev        4.452831
fixedi_HP_dev            2.850216
nonresidential_HP_dev    2.767737
structures_HP_dev        3.426253
equipment_HP_dev         3.635744
intellectual_HP_dev      1.884458
residential_HP_dev       5.827305
dtype: float64

In [146]:
# Standard deviations of first differences of unfiltered series vs gdp 

# Similarly, we get consistent results, changes in comsumption are smaller than in GDP
# Due to consumption smoothing. Investment is far more volatile, especially

data_transp[names_growth].std()/data_transp[names_growth].std()[0]

gdp_diff               1.000000
c_diff                 0.860872
investment_diff        5.137248
fixedi_diff            2.574466
nonresidential_diff    2.524547
structures_diff        3.016171
equipment_diff         3.708367
intellectual_diff      1.602609
residential_diff       5.146236
dtype: float64

In [150]:
# first-order serial correlation of each series in dev from the trend

autocor_dev = pd.DataFrame()
autocor_dev['names'] = names_dev
z = []
for i in names_dev:
    z.append(data_transp[i].autocorr(lag=1))
autocor_dev['Autocorr'] = z
autocor_dev

,names,Autocorr
0,gdp_HP_dev,0.848739
1,c_HP_dev,0.810225
2,investment_HP_dev,0.782248
3,fixedi_HP_dev,0.879093
4,nonresidential_HP_dev,0.876596
5,structures_HP_dev,0.886734
6,equipment_HP_dev,0.838556
7,intellectual_HP_dev,0.907789
8,residential_HP_dev,0.885209


In [152]:
# first-order serial correlation of each series in first differences

autocor_dev = pd.DataFrame()
autocor_dev['names'] = names_growth
z = []
for i in names_growth:
    z.append(data_transp[i].autocorr(lag=1))
autocor_dev['Autocorr'] = z
autocor_dev    

,names,Autocorr
0,gdp_diff,0.364728
1,c_diff,0.072047
2,investment_diff,0.218004
3,fixedi_diff,0.527896
4,nonresidential_diff,0.450968
5,structures_diff,0.464521
6,equipment_diff,0.335472
7,intellectual_diff,0.679592
8,residential_diff,0.540201
